In [2]:
SGA = SymmetricGroupAlgebra(QQ,3)

In [3]:
SGA.dft()

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [9]:
SGA_F7 = SymmetricGroupAlgebra(GF(7),3)

In [10]:
SGA_F7.dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [19]:
SGA_F7.specht_module(Partition([1,1,1]))

Specht module of [(0, 0), (1, 0), (2, 0)] over Finite Field of size 7